In [24]:
import os, shutil
import rasterio as rio
import numpy
from matplotlib import pyplot
from rasterio.plot import show
import numpy as np

In [8]:
loc_orthos = "../EPCExtent_30cm/Orthos/"
loc_ndvi = "../EPCExtent_30cm/Orthos/NDVI"
os.makedirs(loc_ndvi, exist_ok=True)
orthos_files = [os.path.join(loc_orthos,f) for f in os.listdir(loc_orthos) if f.endswith(".tif")]

In [71]:
count = 0
for file in orthos_files:
    count+=1
    ofile = os.path.join(loc_ndvi, os.path.basename(file).replace(".tif","_NDVI.tif"))
    if os.path.exists(ofile):
        continue
    with rio.open(file) as ras:
        print(f"{count} - Starting file {ofile}")
        red = ras.read(1).astype(rio.float32)
        green = ras.read(2).astype(rio.float32)
        blue =  ras.read(3).astype(rio.float32)
        nir = ras.read(4).astype(rio.float32)
        sumzero = red+green+blue+nir
        
        # Potential for valid areas to have 0 values in Red and NIR bands. In these cases we'd get a division by zero error. Mitigate against this.
        ndvi = np.divide(nir-red,nir+red, out=np.zeros_like(nir), where=nir+red!=0)
        
        kwargs = ras.profile
        kwargs.update(
            count=1,
            dtype=rio.float32,
            nodata=-9999
        )
        
        ndvi = np.where(sumzero == 0, -9999, ndvi)
        with rio.open(ofile, 'w', **kwargs) as oras:
            oras.write(ndvi,1)
    
print(f"Finished writing all NDVI files to {loc_ndvi}")

Finished writing all NDVI files to ../EPCExtent_30cm/Orthos/NDVI
